<a href="https://colab.research.google.com/github/smomtahe/Brain_Tumor_Project/blob/main/Brain_Tumor_Vision_Transformer_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install tensorflow transformers

In [15]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForImageClassification, ViTImageProcessor
import numpy as np
import zipfile
import os
from pathlib import Path
from tensorflow.keras.preprocessing import image

# The zip file:
zip_file_path = "/content/MRIBrainTumor.zip"
extracted_dir = "/content/extracted_data"

# Extract the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Function to load images and labels
def load_images_and_labels(image_dir, label):
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
    images, labels = [], []
    for img_path in image_dir.glob("*jpg"):
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        processed_image = processor(images=img_array, return_tensors="tf").pixel_values
        images.append(processed_image)
        labels.append(label)
    return np.vstack(images), labels

In [16]:
# Load data
no_tumor_path = Path(extracted_dir + "/Training/no_tumor")
pituitary_tumor_path = Path(extracted_dir + "/Training/pituitary_tumor")

no_tumor_images, no_tumor_labels = load_images_and_labels(no_tumor_path, 0)
pituitary_tumor_images, pituitary_tumor_labels = load_images_and_labels(pituitary_tumor_path, 1)

# Combine and split the data
x_train, y_train = np.concatenate([no_tumor_images, pituitary_tumor_images]), no_tumor_labels + pituitary_tumor_labels
y_train = to_categorical(y_train, num_classes=2)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [17]:
# Define and compile the Vision Transformer model
def build_vit_classifier():
    input = Input(shape=(224, 224, 3), name='input_image', dtype='float32')
    vit_model = TFAutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=2)

    # Since we're using the model directly, we don't need a custom Model/Sequential
    vit_model.compile(optimizer=Adam(learning_rate=5e-5),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    return vit_model

vit_classifier = build_vit_classifier()

# Train the model
vit_classifier.fit(x_train, y_train, epochs=3, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
loss, accuracy = vit_classifier.evaluate(x_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
31/31 [==============================] - 87s 1s/step - loss: 2.6691 - accuracy: 0.0706 - val_loss: 8.3806 - val_accuracy: 0.0490
Epoch 2/3
31/31 [==============================] - 38s 1s/step - loss: 7.0250 - accuracy: 0.0256 - val_loss: 0.0583 - val_accuracy: 0.0163
Epoch 3/3
8/8 [==============================] - 3s 414ms/step - loss: 0.0777 - accuracy: 0.0082
Loss: 0.07768254727125168, Accuracy: 0.008163264952600002


In [18]:
vit_classifier.save('vit_model')

In [19]:
from tensorflow.keras.models import load_model

model = load_model('vit_model')

In [28]:
from transformers import ViTImageProcessor
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the model
model = load_model('vit_model')


1/1 [==============================] - 2s 2s/step


In [31]:
# Load and preprocess the image
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
img = image.load_img("/content/brain.jpg", target_size=(224, 224))
image_array = image.img_to_array(img)
processed_image = processor(images=image_array, return_tensors="tf").pixel_values

# Ensure the input is correctly formatted
# Note: Depending on how the model was saved and loaded,
# you might need to adjust this to match the expected input format.
predictions = model.predict({'pixel_values': processed_image})

# Assuming a binary classification (tumor, no tumor)
predicted_class_index = np.argmax(predictions, axis=-1)


# Check if predicted_class_index is a scalar or an array and handle accordingly
if np.isscalar(predicted_class_index):
    predicted_class_label = class_labels[predicted_class_index]
else:
    # For multiple predictions, this would iterate over each prediction
    predicted_class_label = [class_labels[i] for i in predicted_class_index]

print(f"Predicted class: {predicted_class_label}")

1/1 [==============================] - 0s 165ms/step
Predicted class: no tumor
